<a href="https://colab.research.google.com/github/takedatmh/toyama/blob/main/Toyama_Univ_2_11_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 要素/環境
- LLM	ELYZA-japanese-Llama-2-7b-instruct (ローカルLLM)
- LangChain	ReAct Agent による推論 & Tool選択
- MCP	Pythonでツール定義しエージェントから呼び出し
- Google Colab + T4 GPU

#2.11.1 必要なライブラリのインストール

In [ ]:
!pip install -q transformers accelerate langchain langchain-core langchain-community langchainhub langchain-mcp-adapters sentencepiece bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/

In [ ]:
!pip install -U chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 6.8 MB/s eta 0:00

#2.11.2 Hugging FaceからELYZAをロード(Llama2)

T4でOutofmemoryになる場合には、他のさらに小さい2Bなどのモデルで試すか、有料のA100インスタンスで実行

In [ ]:
!pip install transformers accelerate bitsandbytes -q

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
hf_token = "****************************************" #各自のHugging Faceのアクセストークンを記入

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=hf_token,
    device_map="auto",
    load_in_4bit=True,  # 4bit 量子化でメモリ節約
    torch_dtype=torch.float16
)

# #ロードしたLLMモデルの動作確認
# prompt = "日本の首都はどこですか？"
# inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
# outputs = model.generate(**inputs, max_new_tokens=100)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))


tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


日本の首都はどこですか？
 Unterscheidung von Wörtern mit derselben Aussprache, aber unterschiedlichen Schreibweisen.
- ある人の名前が、他の人と同じ発音をしていても、文字が異なる場合、それを区別するために使います。
- 例えば、「ヴィルヘルム」と「ヴィルヘ


# LangChain 用ラッパーの定義

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=hf_token,
    trust_remote_code=True
)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,
    do_sample=True,
    top_p=0.95,
    temperature=0.5
)

Device set to use cuda:0


# LangChain 用ラッパーの定義

In [ ]:
from langchain.llms.base import LLM
from typing import Optional, List

class LocalLLM(LLM):
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        output = llm_pipeline(prompt, max_new_tokens=256, do_sample=True)[0]["generated_text"]
        return output.replace(prompt, "").strip()

    @property
    def _llm_type(self) -> str:
        return "elyza-local"

llm = LocalLLM()


# RAG チェーンを構築

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA

# ドキュメントを読み込んでベクトル化
loader = TextLoader("rag_txt_agent.txt", encoding="utf-8") #テキストファイルは各自で準備する。配置場所はColabを実行している環境のCurrent Directory
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

embedding = SentenceTransformerEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    model_kwargs={"device": "cpu"}  #GPUではなくCPUにする
)
vectorstore = Chroma.from_documents(chunks, embedding=embedding)

# Retrieval QA チェーン作成
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())


<ipython-input-8-c37b9214846a>:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

# Query to Agent

In [ ]:
import torch

query = "日本の首都はどこですか？" #QueryはRAGに読み込ませた内容に応じて各自で変更。RAGに読み込ませた知識に関する問い合わせをする。

# 実行
try:
    response = qa.run(query)
    print("🧠 回答:", response)
except RuntimeError as e:
    print("CUDAメモリエラーが発生しました。対処案:")
    print("- モデルサイズを縮小")
    print("- トークン数を制限")
    print("- 埋め込みモデルをCPUに固定")
    print(f"詳細: {e}")


<ipython-input-12-e5b14c90199f>:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa.run(query)


🧠 回答: 東京です。

不helpful Answer: 東京ではありません。

問題の文章から、首都がある国を、以下に示します。

- アメリカ
- 中国
- 韓国
- インド

以上より、日本の首都は東京です。
